# C/C++ __stdcall DLL, Excel VBA

## 1 C/C++ __stdcall DLL 

  For `Visual Basic` applications (or applications in other languages such as Pascal or Fortran) to call functions in a C/C++ DLL, the functions must be exported using the correct calling convention without any name decoration done by the compiler.
  
VBA can **only** call `__stdcall` functions, not `__cdecl` functions. 
  
*  `__stdcall` creates the correct calling convention for the function (the called function cleans up the stack and parameters are passed from right to left)

* ` __declspec(dllexport)` is used on an `exported` function in a DLL

Below is the example of techniques which facilitate the use of use of MinGW to create DLLs, exporting functions which may be called from Visual Basic Application with Excel. 

Example: Equations for Region4 of [IAPWS-IF97](http://www.iapws.org/relguide/IF97-Rev.pdf)

* 8 Equations for Region 4 

  * 8.1 The Saturation-Pressure Equation (Basic Equation) P33,Eq30
  
  * 8.2 The Saturation-Temperature Equation (Backward Equation) P35, Eq31


### 1.1 Create your DLL with `__stdcall` calling convention

Create a DLL with the following code:

* region4.h

* region4.c

For Windows,export all functions as `__stdcall`.

```c
#define DLLPORT __declspec(dllexport) double __stdcall 
```
**NOTE**： The following header `region4.h`  declares the interface for 

* building the **Windows/Linux** shared library  

* building an executable that uses the  shared library.


In [ ]:
%%file ./demo/src/region4.h

#pragma once

#ifdef __cplusplus
extern "C" {
#endif

#ifdef BUILD_DLL

    #ifdef WIN32
        #define PORT __declspec(dllexport) double __stdcall 
    #else
        #define PORT double
    #endif    

#else

    #ifdef WIN32
        #define PORT __declspec(dllimport) double __stdcall   
    #else
        #define PORT double
    #endif    

#endif

PORT pSat(double T);
PORT TSat(double p);
        
#ifdef __cplusplus
	}
#endif        

When you create header files for your DLLs, use

* ` __declspec(dllexport) ` adds the `export` directive to the object fileworks

* ` __declspec(dllimport)`  on the declarations of the public symbols

In [ ]:
%%file ./demo/src/region4.c
#include <math.h>
#include "region4.h"

//
//  Initialize coefficients for region 4
//
static double n[11] = {0, 0.11670521452767E+04, -0.72421316703206E+06, -0.17073846940092E+02,
                          0.12020824702470E+05, -0.32325550322333E+07, 0.14915108613530E+02,
                         -0.48232657361591E+04, 0.40511340542057E+06, -0.23855557567849E+00,
                          0.65017534844798E+03};

PORT pSat(double T)
// saturation pressure of water
// pSat in MPa
// T :temperaturein K
//
// pSat = -1: temperature outside range
//
{
    double pS;
    if (T < 273.15 || T > 647.096) // tc_water=647.096
        pS = -1.0;
    else
    {
        double del = T + n[9] / (T - n[10]);
        double aco = del * (del + n[1]) + n[2];
        double bco = del * (n[3] * del + n[4]) + n[5];
        double cco = del * (n[6] * del + n[7]) + n[8];
        pS = pow(2 * cco / (-bco + sqrt(bco * bco - 4 * aco * cco)), 4);
    }
    return pS;
}

PORT TSat(double p)
// saturation temperature of water
// TSat in K
// p :pressure MPa
//
// TSat=-1: pressure outside range
//
{
    double TS;
    if (p < 0.000611212677 || p > 22.064)
        TS = -1.0;
    else
    {
        double bet = pow(p, 0.25);
        double eco = bet * (bet + n[3]) + n[6];
        double fco = bet * (n[1] * bet + n[4]) + n[7];
        double gco = bet * (n[2] * bet + n[5]) + n[8];
        double dco = 2.0 * gco / (-fco - sqrt(fco * fco - 4.0 * eco * gco));
        TS = 0.5 * (n[10] + dco - sqrt((n[10] + dco) * (n[10] + dco) - 4.0 * (n[9] + n[10] * dco)));
    }
    return TS;
}

In [ ]:
!gcc -c -DBUILD_DLL -o ./demo/obj/region4.o ./demo/src/region4.c -I./demo/src
!gcc -shared -o ./demo/bin/libregion4.dll ./demo/obj/region4.o  -Wl,--add-stdcall-alias

* -DBUILD_DLL:
   
  * -Dname: `Predefine name as a macro`, with definition 
  

*  -Wl,option 

   Pass **option** as an option to the **linker**. If option contains commas, it is split into multiple options at the commas.


* --add-stdcall-alias:
   
   This adds an undecorated alias for the `exported function names` that is simply **the name of the function** 

### 1.2  Add -static-libgcc ,-output-def=libregion4.def

 Links the GNU `libgcc` library `statically`

In [1]:
!gcc -c -DBUILD_DLL -o ./demo/obj/region4.o ./demo/src/region4.c -I./demo/src
!gcc -shared -o ./demo/bin/libregion4.dll -static-libgcc ./demo/obj/region4.o -Wl,--add-stdcall-alias,-output-def=./demo/bin/libregion4.def

* -static-libgcc

   This option links the GNU `libgcc` library **statically** 
   

* -output-def=libregion4.def

    Name of `.def` file to be created.
    
    **def:** A module-definition  file is a text file containing one or more module statements that describe various attributes of a DLL

In [ ]:
%load ./demo/bin/libregion4.def

In [ ]:
%%file ./demo/makefile-region4

CC=gcc
CFLAGS=-DBUILD_DLL

SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

all: libregion4.dll

libregion4.dll: obj
	 $(CC) -shared -o $(BINDIR)libregion4.dll -static-libgcc $(OBJDIR)region4.o -Wl,--add-stdcall-alias,-output-def=libregion4.def
	 del .\demo\obj\region4.o
    
obj: 
	 $(CC) -c $(CFLAGS) -o $(OBJDIR)region4.o $(SRCDIR)region4.c
     
clean:
	 del .\demo\bin\libregion4.dll

In [ ]:
!make -f ./demo/makefile-region4

### 1.3 Call Dll from C/C++

In [2]:
%%file ./demo/src/mainReg4.c

#include <stdio.h> 
#include "region4.h"

int main() {
     double T=300.0;
     printf("Saturation P is %f\n", pSat(T));  
     return 0;
}

Overwriting ./demo/src/mainReg4.c


In [3]:
!gcc -c -o ./demo/obj/mainReg4.o ./demo/src/mainReg4.c 
!gcc -o  ./demo/bin/mainReg4 ./demo/obj/mainReg4.o -I./demo/src/ -L./demo/bin/ -lregion4

In [4]:
!.\demo\bin\mainReg4

Saturation P is 0.003537


### 1.4 Python API

* `__stdcall` calling convention: **windll, WINFUNCTYPE**

In [9]:
%%file ./demo/src/region4.py

from ctypes import windll,c_double,WINFUNCTYPE

flib = windll.LoadLibrary('./demo/bin/libregion4.dll')
prototype = WINFUNCTYPE(c_double,c_double)

def pSat(T):
    f = prototype(("pSat", flib),)
    return f(T)

def TSat(p):
    f = prototype(("TSat", flib),)
    return f(p)

Overwriting ./demo/src/region4.py


**add `mathfuns.py` into the interperte search path**

In [6]:
import sys
sys.path.append('./demo/src')

In [15]:
%%file ./demo/src/test4.py

import unittest
from region4 import *

class Region4Test (unittest.TestCase):

    def setUp(self):
        # IF97-dev,Table35 Page 34 : T(K) p(MPa)
        self.tab35=[[300, 0.353658941e-2],
                    [500, 0.263889776e1],
                    [600, 0.123443146e2]]

        # IF97-dev, Table 36 Page 36 :  p(MPa) T(K)
        self.tab36=[[0.1, 0.372755919e3],
                    [  1, 0.453035632e3],
                    [ 10, 0.584149488e3]]

    def test_pSat(self):
        places = 6
        for item in  self.tab35:
             self.assertAlmostEqual(pSat(item[0]),item[1],places)

    def test_TSat(self):
        places = 6
        for item in  self.tab36:
             self.assertAlmostEqual(TSat(item[0]),item[1],places)
     
if __name__ == '__main__':
    unittest.main()       

Overwriting ./demo/src/test4.py


In [16]:
%run ./demo/src/test4.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


## 2 Call DLL from Excel VBA(64bits)

Do as the following steps:

### 2.1 Put  DLL in the default path of Windows'dll 


`libregion4.dll`  in `C:\windows\system`


### 2.2 Create the xlsm

`demo-r4.xlsm` in `./demo/ExcelVBA/`


### 2.3 Create the `VBA module` to call the DLL.

There are a few ways to open up the **VBA Editor** in Excel.

* From the `Developer Tools` tab, you can click the `Visual Basic` button.

* A keyboard shortcut: press "Alt+F11" :

![vba](./img/vba.jpg)

Into **VBA** Editor, then， create the module **mathfuns** to call library:

```VBA
Declare PtrSafe Function pSat Lib "libregion4" (ByVal T As Double) As Double
Declare PtrSafe Function TSat Lib "libregion4" (ByVal p As Double) As Double

Public Function CalpSat(ByVal T As Double) As Double
    CalpSat = pSat(T)
End Function
 
Public Function  CalTSat(ByVal p As Double) As Double
    CalTSat = TSat(p)
End Function
```

### 2.4  call VBA methods in cells

![demo-r4](./img/demo-r4.jpg)

## 3 Excel4Engineering

Examples of Excel VBA for Engineering: Analysizing the Ideal Rankine Cycle, Monitoring Industrial Process 

https://github.com/thermalogic/Excel4Engineering

## Reference

###  C/C++

* ctypes http://docs.python.org/3/library/ctypes.html
    
###  Microsoft Excel

* Excel VBA Programming:  http://www.homeandlearn.org/the_excel_vba_editor.html

* [Language reference for Visual Basic for Applications(VBA)](https://docs.microsoft.com/en-us/office/vba/api/overview/language-reference)

* [Excel VBA reference](https://docs.microsoft.com/en-us/office/vba/api/overview/excel)

* [Excel add-in tutorial](https://docs.microsoft.com/en-us/office/dev/add-ins/tutorials/excel-tutorial)

* DLLs in Visual C++ https://msdn.microsoft.com/en-us/library/1ez7dh12.aspx

* Calling DLL Functions from Visual Basic Applications https://msdn.microsoft.com/en-us/library/dt232c9t.aspx


